## Taking a brief look at the fire dataset

In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
from datetime import datetime, timedelta
import os
import numpy as np

In [2]:
df = pd.read_csv("dataset/fire_archive_M6_96619.csv")
df.sort_values("acq_date").head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-11.8070,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0
285,-14.2000,130.6596,324.2,2.0,1.4,2019-08-01,524,Aqua,MODIS,42,6.3,301.8,28.8,D,0
284,-14.3457,129.8308,321.8,1.7,1.3,2019-08-01,524,Aqua,MODIS,57,6.3,301.9,18.7,D,0
283,-14.2476,130.6929,321.2,2.0,1.4,2019-08-01,524,Aqua,MODIS,61,6.3,303.7,21.0,D,0
282,-14.2568,130.7132,325.2,2.0,1.4,2019-08-01,524,Aqua,MODIS,54,6.3,304.4,30.7,D,0


In [3]:
df.sort_values("acq_date").tail()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
35566,-27.4541,149.4632,329.1,1.2,1.1,2019-09-30,407,Aqua,MODIS,70,6.3,315.2,14.2,D,0
35567,-27.4063,151.6380,334.3,1.6,1.3,2019-09-30,407,Aqua,MODIS,72,6.3,314.2,29.9,D,0
35568,-27.4175,151.6407,331.5,1.6,1.3,2019-09-30,407,Aqua,MODIS,64,6.3,317.6,22.2,D,0
35570,-22.4856,150.7192,327.5,1.8,1.3,2019-09-30,408,Aqua,MODIS,0,6.3,299.7,40.2,D,0
36010,-30.8641,121.4995,309.1,1.8,1.3,2019-09-30,1645,Aqua,MODIS,77,6.3,290.9,23.0,N,2


## Clean the data frame
We only need lat, long, date, and bright-ti4 for our visualization

In [4]:
df.drop(['scan', 'track', 'acq_time', 'satellite', 'instrument', 'version', 'brightness', 'bright_t31', 'frp', 'type'], axis=1, inplace=True)

max_conf = df['confidence'].max()
min_conf = df['confidence'].min()
print(max_conf)
print(min_conf)

100
0


In [5]:
df.head()

,latitude,longitude,acq_date,confidence,daynight
0,-11.8070,142.0583,2019-08-01,48,D
1,-11.7924,142.0850,2019-08-01,71,D
2,-12.8398,132.8744,2019-08-01,42,D
3,-14.4306,143.3035,2019-08-01,33,D
4,-12.4953,131.4897,2019-08-01,36,D


## Split the data into days & scale weights for heatmap

In [6]:
days = []
for month in range(8, 10):
    for day in range(1, 32):
        daystr = f'2019-0{month}-{day}'
        if day < 10:
            daystr = f'2019-0{month}-0{day}'

        d = df[df["acq_date"] == daystr][["latitude", "longitude", "confidence"]]
        days.append(d)

days.pop()

,latitude,longitude,confidence


In [7]:
print(days[-1])

       latitude  longitude  confidence
35342  -11.6667   142.0859          40
35343  -11.6693   142.1066          27
35344  -11.8285   142.2454          44
35345  -11.8845   142.2604          51
35346  -11.8802   142.2552          44
...         ...        ...         ...
36006  -28.0656   131.6334          85
36007  -28.2625   128.5494          27
36008  -28.2941   128.5123          43
36009  -28.3001   128.5530          41
36010  -30.8641   121.4995          77

[669 rows x 3 columns]


Prepare the data for the heatmap

In [8]:
def scale(x, min, max):
    return (x - min) / (max - min)

data = []
for d in days:
    scaled = d.copy()
    scaled['confidence'] = scaled['confidence'].apply(scale, args=(min_conf, max_conf))
    data.append(scaled.values.tolist())

data[0]

[[-11.807, 142.0583, 0.48],
 [-11.7924, 142.085, 0.71],
 [-12.8398, 132.8744, 0.42],
 [-14.4306, 143.3035, 0.33],
 [-12.4953, 131.4897, 0.36],
 [-12.6191, 142.1998, 0.68],
 [-14.3655, 143.5682, 0.24],
 [-14.3195, 143.5198, 0.79],
 [-13.1654, 141.9715, 0.72],
 [-11.5473, 132.6796, 0.4],
 [-11.5417, 132.649, 0.42],
 [-11.5471, 132.6538, 0.65],
 [-12.7626, 142.1759, 0.88],
 [-12.705, 133.907, 0.57],
 [-13.1092, 134.1848, 0.67],
 [-14.0539, 142.3327, 0.52],
 [-12.6996, 133.9138, 0.55],
 [-12.7499, 134.5251, 0.66],
 [-12.7141, 134.3991, 0.64],
 [-12.7061, 134.4077, 0.64],
 [-13.8656, 141.9771, 0.69],
 [-12.6491, 134.3953, 0.79],
 [-13.1015, 134.1926, 0.55],
 [-12.6703, 142.1102, 0.78],
 [-13.3042, 142.04, 0.76],
 [-13.2236, 141.9914, 0.69],
 [-13.2821, 142.4617, 0.19],
 [-13.2159, 142.002, 0.52],
 [-13.2537, 142.5242, 0.37],
 [-12.4513, 141.8071, 0.68],
 [-12.4527, 141.8162, 0.83],
 [-12.544, 142.4455, 0.57],
 [-12.463, 141.8241, 0.51],
 [-12.5567, 141.8979, 0.88],
 [-12.5581, 141.9071, 0.5

Create the time index so that folium knows what date to put on each step

In [9]:
start_date = datetime(year=2019, month=8, day=1)
time_index = [(start_date + k * timedelta(1)).strftime("%Y-%m-%d") for k in range(len(data))]

Make a heatmap (lol) of brightness over coordinates

In [10]:
m = folium.Map(
    [-24.0, 140.0],
    zoom_start=4,
)

hm = HeatMapWithTime(data, index=time_index, auto_play=True)
hm.add_to(m)

os.makedirs('graphs/', exist_ok=True)
m.save('graphs/fire_archive_M6_96619.html')